# Analisis de información den BSC Exito

In [1]:
# importar librerias
import os
import pandas as pd
import yaml
import plotly.express as px


In [2]:
# leyendo archivo de configuración
with open("insumos/config.yml", "r",encoding='utf-8') as ymlfile:
    config = yaml.safe_load(ymlfile)


Aqui vamos cargar los datos del archivo de BSC del exito y realizar algunas transformaciones para el analisis de la información

In [3]:
#leyendo data del bsc
ruta_archivo = os.path.join(os.getcwd(),'insumos',config['datos']['balanced_score'])
col_usar = config['balanced_score_columnas'].keys()
tipado_col = {nomcol: tipo[0]  for nomcol, tipo in config['balanced_score_columnas'].items()}
nombre_col = [tipo[1] for tipo in config['balanced_score_columnas'].values()]
data = pd.read_excel(ruta_archivo,usecols=col_usar,dtype=tipado_col)
data.columns = nombre_col

In [4]:
# informacion general de los datos.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159942 entries, 0 to 159941
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   mes                159942 non-null  int32  
 1   region             158827 non-null  object 
 2   formato            159942 non-null  object 
 3   negocio            159942 non-null  object 
 4   marca              159942 non-null  object 
 5   producto           159942 non-null  object 
 6   ean                159942 non-null  object 
 7   codigo_sap         159942 non-null  object 
 8   peso               159942 non-null  float64
 9   plu                4865 non-null    object 
 10  venta_cop_año_ant  150343 non-null  float64
 11  venta_cop_año_act  13696 non-null   float64
 12  venta_un_ant       150343 non-null  float64
 13  venta_un_act       13696 non-null   float64
dtypes: float64(5), int32(1), object(8)
memory usage: 16.5+ MB


Ahora cargaremos la informacion de los margenes a diferentes detalles:
* Margen por negocio
* Margen por marca
* Margen por material
Se cargaran en tablas separadas

In [5]:
def dfarchivoAFO(ruta:str,sheet_name:str,nombrecol:dict): 
      '''
      Lee un archivo de excel que contiene una tabla extraida de AFO.
      ARG: ruta: str
            sheet_name : str: nombre de la hoja
            nombre_col : dict: con el nombre y tipo de dato de las columnas
      return : data frame
      '''      
      df = pd.ExcelFile(ruta)
      df = df.parse(sheet_name,dtype=str) 
      df.columns = nombrecol.keys()      
      df = df.astype(nombrecol)
      return df
     

In [6]:
ruta_margen = os.path.join(os.getcwd(),'insumos',config['datos']['margen'])

lista_hojas = []
for hojas in config['config_margen'].keys():
    lista_hojas.append(hojas)
margen_sector = dfarchivoAFO(ruta_margen,lista_hojas[0],config['config_margen'][lista_hojas[0]])
margen_marca = dfarchivoAFO(ruta_margen,lista_hojas[1],config['config_margen'][lista_hojas[1]])
margen_material = dfarchivoAFO(ruta_margen,lista_hojas[2],config['config_margen'][lista_hojas[2]])



## Realizando primeros analisis descriptivos.
Analizaremos el comportamiento general por mes, por negocio y por marca, para identificar las oportunidades de crecimiento o marcas potenciales.

In [7]:
# Agrupando variables
lista_variables_agrupa_general = ['ean','plu','peso','venta_cop_año_act','venta_un_act','formato'] # lista con
data_ajustada = data.copy()
for col in lista_variables_agrupa_general:
    del data_ajustada[col]

In [8]:
## hay valores nulos en algunas variables de ventas, por lo tanto esto se llenaran con 0
data_ajustada['venta_cop_año_ant'] = data_ajustada['venta_cop_año_ant'].fillna(0)
data_ajustada['venta_un_ant'] = data_ajustada['venta_un_ant'].fillna(0)

In [9]:
# filtramos informacion no relevante
data_ajustada= data_ajustada[~data_ajustada['region'].isna()]# eliminamos los registros que no tienen region. 
data_ajustada= data_ajustada[data_ajustada['negocio']!='Otros Oper Cciales'] # eliminamos el negicio de Otros Oper Cciales.
data_ajustada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158822 entries, 0 to 159941
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   mes                158822 non-null  int32  
 1   region             158822 non-null  object 
 2   negocio            158822 non-null  object 
 3   marca              158822 non-null  object 
 4   producto           158822 non-null  object 
 5   codigo_sap         158822 non-null  object 
 6   venta_cop_año_ant  158822 non-null  float64
 7   venta_un_ant       158822 non-null  float64
dtypes: float64(2), int32(1), object(5)
memory usage: 10.3+ MB


In [17]:
data_ajustada.groupby(['mes','region'])[['venta_cop_año_ant','venta_cop_año_ant']].sum().reset_index()

,mes,region,venta_cop_año_ant,venta_cop_año_ant
0,1,R Barranquilla,346970.39,346970.39
1,1,R Bogota norte,1148266.71,1148266.71
2,1,R Bucaramanga,94630.23,94630.23
3,1,R Cali,237737.80,237737.80
4,1,R Cartagena,3323.11,3323.11
...,...,...,...,...
91,12,R Cali,304488.61,304488.61
92,12,R Cartagena,6652.22,6652.22
93,12,R Ibague,85332.81,85332.81
94,12,R Medellin,724929.63,724929.63


In [10]:
#imprmiendo valores unicos de las variables ccategorias
list_no_mostrar_valores = ['producto','codigo_sap']
for col in data_ajustada.columns:
    if data_ajustada[col].dtype =='object' and col not in list_no_mostrar_valores :
        print(col)
        print(data_ajustada[col].unique())



region
['R Pereira' 'R Cali' 'R Bucaramanga' 'R Ibague' 'R Barranquilla'
 'R Medellin' 'R Bogota norte' 'R Cartagena']
negocio
['Chocolates' 'Galletas' 'Pastas' 'Café' 'Emprendimientos' 'Carnico'
 'Culinarios' 'Nutricion experta']
marca
['La Especial' 'KIBO' 'Doria' 'Monticello' 'Naturela' 'Matiz' 'Chocolisto'
 'Evok' 'Zenú' 'Tosh' 'Comarrico' 'Badia' 'Bénet' 'Dux' 'Noel' 'Festival'
 'Drácula' 'Saltin Noel' 'Gol' 'Jumbo' 'Jet' 'MontBlanc' 'Corona' 'Tikys'
 'Chocolate La Especial' 'Lyne' 'Sello Rojo' 'Colcafé' 'La Bastilla'
 'Ducales' 'Rica' 'Carve' 'Diversa' 'Nutresa' 'BastiYá' 'Corona Harina'
 'Cruz' 'Tesalia' 'Santander']


## Análisis gráfico variables categóricas principales

### Analisis por Negocio

El primer gráfico compara las ventas de por negocio durante los meses del año
la idea es poder identificar tendencias crecientes o decrecientes que permitan hacer zoom sobre determinado negocio.

In [12]:

variables_a_agrupar= ['mes','negocio'] # variables a agrupar
variables_numericas ={'venta_cop_año_ant':'sum','venta_un_ant':'sum'}
ventas_por_negocio_sorted = data_ajustada.groupby(variables_a_agrupar).agg(variables_numericas).reset_index()
ventas_por_negocio_sorted = ventas_por_negocio_sorted.sort_values('mes', ascending=True)

# Gráfico de lineas por mes y negocio
fig = px.line(
    ventas_por_negocio_sorted,
    x='mes',
    y='venta_cop_año_ant',
    color='negocio',  # Cada negocio una línea distinta
    markers=True,     # Opcional: puntos en cada mes
    line_shape='linear',  # Forzamos línea continua (opcional, pero ayuda)
)
fig.update_layout(
    title='Tendencia de Ventas por Negocio y Mes (Año Anterior)',
    xaxis_title='Mes',
    yaxis_title='Ventas (COP)',
    height=700,
    width=1100,
    template='plotly_white',
    legend_title='Negocio',
)
fig.show()

Analicemos los margenes

In [17]:
margen_sector['marge_real_label'] = margen_sector['marge_real'].round(2)
fig = px.bar(
    margen_sector,
    x='negocio',
    y='marge_real_label',
    color='negocio',
    text='marge_real_label'  # Mostrar el valor de marge_real como etiqueta
)

# Ajustes estéticos
fig.update_traces(textposition='outside')  # Coloca las etiquetas por fuera de las barras
fig.update_layout(
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    title='Margen por Negocio',
    xaxis_title='Negocio',
    yaxis_title='Margen Real',
    showlegend=False  # Oculta la leyenda si color y x son la misma variable
)

fig.show()

Calculamos algunas medidas descriptivas de las ventas por negocio en determinado tiempo

In [ ]:
## resumen por negocio
ventas_por_sector_sorted = ventas_por_negocio_sorted.groupby(['negocio']).agg(
    total_ventas = ('venta_cop_año_ant','sum'),
    media_ventas = ('venta_cop_año_ant','mean'),
    desviacion_ventas = ('venta_cop_año_ant','std'),
    mediana_ventas = ('venta_cop_año_ant','median'),
    minimo_ventas = ('venta_cop_año_ant','min'),
    maximo_ventas = ('venta_cop_año_ant','max')    
).reset_index().sort_values('total_ventas',ascending=False)

ventas_por_sector_sorted = pd.merge(ventas_por_sector_sorted,margen_sector[['negocio','marge_real_label']], on = ['negocio'], how='left')
ventas_por_sector_sorted.sort_values(by='total_ventas', ascending=False)

,negocio,total_ventas,media_ventas,desviacion_ventas,mediana_ventas,minimo_ventas,maximo_ventas,marge_real_label
0,Galletas,9481142.21,790095.184167,110388.154270,755638.875,718346.34,1125054.03,44.68
1,Chocolates,8897315.78,741442.981667,61802.242991,737819.385,644256.92,858805.33,40.19
2,Pastas,7159625.00,596635.416667,28535.732794,599943.000,546192.00,658821.00,47.16
3,Café,4386358.00,365529.833333,29269.031387,361352.500,326356.00,431936.00,33.37
4,Carnico,2552216.00,212684.666667,26666.143125,207986.500,173571.00,257836.00,32.36
5,Culinarios,478699.00,39891.583333,8601.830603,40672.000,29337.00,60319.00,NaN
6,Nutricion experta,68180.00,5681.666667,689.199580,5620.000,4678.00,7117.00,NaN
7,Emprendimientos,46437.00,3869.750000,943.116220,3600.500,2965.00,6416.00,41.63


### Exploremos ahora analisis por marca

In [18]:
variables_a_agrupar= ['marca'] # variables a agrupar
variables_numericas ={'venta_cop_año_ant':'sum', 'codigo_sap': pd.Series.nunique}
ventas_por_marca_sorted = data_ajustada.groupby(variables_a_agrupar).agg(variables_numericas).reset_index()
ventas_por_marca_sorted = ventas_por_marca_sorted.sort_values('venta_cop_año_ant', ascending=False)


In [19]:
fig = px.bar(
    ventas_por_marca_sorted,
    x='venta_cop_año_ant',
    y='marca',  # Etiquetas por marca en el eje Y
    orientation='h',
    text='venta_cop_año_ant',  # Mostrar las ventas como texto en las barras
    title='Ventas por Marca - Año Anterior',
    labels={'venta_cop_año_ant': 'Ventas (COP)', 'marca': 'Marca'},
)

# Mejoras visuales
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},  # Asegura que el orden sea descendente en el eje x
    xaxis_title='Ventas (COP)',
    yaxis_title='Marca',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    template='plotly_white',
    margin=dict(l=100, r=40, t=60, b=40)
)

fig.show()

In [187]:
data_ajustada.info()

<class 'pandas.core.frame.DataFrame'>
Index: 158822 entries, 0 to 159941
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   mes                158822 non-null  int32  
 1   region             158822 non-null  object 
 2   negocio            158822 non-null  object 
 3   marca              158822 non-null  object 
 4   producto           158822 non-null  object 
 5   codigo_sap         158822 non-null  object 
 6   venta_cop_año_ant  158822 non-null  float64
 7   venta_un_ant       158822 non-null  float64
dtypes: float64(2), int32(1), object(5)
memory usage: 10.3+ MB


### revisar margenes de las marcas

In [22]:
## funcion para calcular limites de valores atipicos.
def valores_atipicos(df,columna):
    '''
    Permite exlcuir valores atipicos utilizando los quartiles
    arg: df: data frame
            columna str.  valor de la columna que se desea eliminar atipico
    '''
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df = df[(df[columna] >= limite_inferior) & (df[columna] <= limite_superior)]
    return df

In [23]:
marcas_inciales = set(margen_marca['marca'])

In [24]:
marge_bruto_ajus_marcas = valores_atipicos(margen_marca,'margen_real')
marcas_resultantes = set(marge_bruto_ajus_marcas['marca'])


In [31]:
margen_marca.describe()

,margen_real,margen_ppto
count,66.000000,46.000000
mean,587.899709,41.068297
std,4285.248493,17.370471
min,-47.821635,6.336921
25%,32.475341,33.222109
50%,42.952196,39.121450
75%,51.257642,48.105107
max,34866.666667,100.000000


In [25]:
## se muestran las marcas que salieron de la medicion por consderarse atipicas en cuanto a sus margenes
marcas_inciales.difference(marcas_resultantes)

{'Atlantic',
 'CELSIA',
 'Chocolyne',
 'Colanta',
 'Compañías GN',
 'Ecopack',
 'Genérico',
 'Marca Genérica',
 'Nutresa',
 'Recreo Noel',
 'Redoxon',
 'Setas de Cuivá',
 'Sin asignar',
 'Sorbitos',
 'Sura'}

In [26]:
fig = px.box(marge_bruto_ajus_marcas,             
             y = 'margen_real',points="all", title="Distribución del margen (sin outliers)",
             labels={"margen": "Margen financiero"})
fig.show()

In [27]:
marge_bruto_ajus_marcas['margen_real_label'] = marge_bruto_ajus_marcas['margen_real'].round(2)
marge_bruto_ajus_marcas = marge_bruto_ajus_marcas.sort_values(by='margen_real_label',ascending=False)

C:\Users\COscaro\AppData\Local\Temp\ipykernel_8372\1462690053.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [32]:
# analisis  por margen 
fig = px.bar(
    marge_bruto_ajus_marcas,
    y='marca',
    x='margen_real_label',
    color='marca',
    text='margen_real_label',
    orientation='h'  # Barras horizontales
)

# Ajustes estéticos
fig.update_traces(textposition='outside')  # Muestra el texto fuera de las barras
fig.update_layout(
    title='Margen por marca',
    xaxis_title='Margen Real',
    yaxis_title='Marca',
    uniformtext_minsize=8,
    uniformtext_mode='hide',
    showlegend=False,
    height=600
)

fig.show()

In [33]:
ventas_por_marca_sorted['ventas_acumuladas'] = ventas_por_marca_sorted['venta_cop_año_ant'].cumsum()
total_ventas = ventas_por_marca_sorted['venta_cop_año_ant'].sum()
ventas_por_marca_sorted['%_ventas'] = (ventas_por_marca_sorted['venta_cop_año_ant'] / total_ventas) * 100
ventas_por_marca_sorted['%_ventas_acumuladas'] = ventas_por_marca_sorted['%_ventas'].cumsum()
ventas_por_marca_sorted['%_ventas'] = ventas_por_marca_sorted['%_ventas'].round(2)
ventas_por_marca_sorted['%_ventas_acumuladas'] = ventas_por_marca_sorted['%_ventas_acumuladas'].round(2)
ventas_por_marca_sorted = pd.merge(ventas_por_marca_sorted,margen_marca[['marca','margen_real']], on = 'marca', how='left')
ventas_por_marca_sorted

,marca,venta_cop_año_ant,codigo_sap,ventas_acumuladas,%_ventas,%_ventas_acumuladas,margen_real
0,Doria,4740097.00,50,4740097.00,14.33,14.33,50.852965
1,Tosh,2993791.67,116,7733888.67,9.05,23.39,45.308186
2,Saltin Noel,2880805.00,26,10614693.67,8.71,32.10,40.058632
3,Zenú,2462989.00,48,13077682.67,7.45,39.55,32.675798
4,Sello Rojo,2272941.00,22,15350623.67,6.87,46.42,33.951147
5,Corona,2130187.00,39,17480810.67,6.44,52.86,33.574302
6,Ducales,1806073.60,18,19286884.27,5.46,58.32,51.392535
7,Noel,1613320.20,46,20900204.47,4.88,63.20,42.449615
8,Comarrico,1423368.00,19,22323572.47,4.30,67.50,40.241585
9,La Especial,1351523.89,51,23675096.36,4.09,71.59,46.675763


In [34]:
ventas_por_marca_sorted.describe()

,venta_cop_año_ant,codigo_sap,ventas_acumuladas,%_ventas,%_ventas_acumuladas,margen_real
count,3.900000e+01,39.000000,3.900000e+01,39.000000,39.000000,37.000000
mean,8.479480e+05,26.076923,2.726765e+07,2.564103,82.454103,37.946423
std,1.077909e+06,24.956156,7.907568e+06,3.258529,23.911203,17.782205
min,7.000000e+00,1.000000,4.740097e+06,0.000000,14.330000,-47.821635
25%,5.318500e+04,6.000000,2.434498e+07,0.160000,73.615000,33.574302
50%,4.786990e+05,21.000000,3.138770e+07,1.450000,94.910000,40.241585
75%,1.345641e+06,39.000000,3.283958e+07,4.070000,99.300000,46.819016
max,4.740097e+06,116.000000,3.306997e+07,14.330000,100.000000,64.996210


In [36]:
lista_marcas_analizar = ['Chocolisto','Jumbo','Matiz','Dux']
marcas_analizar = data_ajustada[data_ajustada['marca'].isin(lista_marcas_analizar)]

lista_agrupadas = ['mes','marca','producto','codigo_sap']
marcas_analizar = marcas_analizar.groupby(lista_agrupadas).sum('venta_cop_año_ant').reset_index()
marcas_analizar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   mes                958 non-null    int32  
 1   marca              958 non-null    object 
 2   producto           958 non-null    object 
 3   codigo_sap         958 non-null    object 
 4   venta_cop_año_ant  958 non-null    float64
 5   venta_un_ant       958 non-null    float64
dtypes: float64(2), int32(1), object(3)
memory usage: 41.3+ KB


In [37]:
# Gráfico de lineas por mes y negocio

agrupacion_por_marca = marcas_analizar.groupby(['mes','marca']).sum('venta_cop_año_ant').reset_index()

fig = px.line(
    agrupacion_por_marca,
    x='mes',
    y='venta_cop_año_ant',
    color='marca',  # Cada negocio una línea distinta
    markers=True,     # Opcional: puntos en cada mes
    line_shape='linear',  # Forzamos línea continua (opcional, pero ayuda)
)
fig.update_layout(
    title='Tendencia de Ventas por marca y mes (Año Anterior)',
    xaxis_title='Mes',
    yaxis_title='Ventas (COP)',
    height=700,
    width=1100,
    template='plotly_white',
    legend_title='Marca',
)
fig.show()

In [38]:
ventas_mensuales = agrupacion_por_marca.groupby(['marca', 'mes'], as_index=False)['venta_cop_año_ant'].sum()
    
    # Crear un DataFrame resultado
resultados = []
for marca, datos in ventas_mensuales.groupby('marca'):
    datos = datos.sort_values('mes')
    datos = datos.set_index('mes')['venta_cop_año_ant']
    venta_ultimo_mes = datos.to_dict()
    venta_ultimo_mes = {k: venta_ultimo_mes.get(k, 0) for k in range(1, 13)}  # asegurar que estén todos los meses
    venta_diciembre = venta_ultimo_mes[12]
    venta_noviembre = venta_ultimo_mes[11]
    trimestre = [venta_ultimo_mes[9], venta_ultimo_mes[10], venta_ultimo_mes[11]]
    año_completo = [venta_ultimo_mes[m] for m in range(1, 12)]  # enero a noviembre
    var_ult_mes = ((venta_diciembre - venta_noviembre) / venta_noviembre) if venta_noviembre != 0 else None
    prom_año = sum(año_completo) / len(año_completo) if año_completo else 0
    var_prom_año = ((venta_diciembre - prom_año) / prom_año) if prom_año != 0 else None
    prom_trimestre = sum(trimestre) / len(trimestre) if trimestre else 0
    var_trimestre = ((venta_diciembre - prom_trimestre) / prom_trimestre) if prom_trimestre != 0 else None
    resultados.append({
        'marca': marca,
        'ventas_totales': sum(venta_ultimo_mes.values()),
        'var_ultimo_mes': var_ult_mes,
        'var_ultimo_trimestre': var_trimestre,
        'var_prom_anual': var_prom_año
    })
pd.DataFrame(resultados)

,marca,ventas_totales,var_ultimo_mes,var_ultimo_trimestre,var_prom_anual
0,Chocolisto,604061.00,0.141348,0.049502,-0.020626
1,Dux,383952.00,-0.094540,-0.128978,-0.053224
2,Jumbo,592990.99,0.259080,0.004891,-0.018294
3,Matiz,574141.00,0.108687,0.112625,0.168551


In [39]:
ventas_mensuales = agrupacion_por_marca.groupby(['marca', 'mes'], as_index=False)['venta_cop_año_ant'].sum()
    
    # Crear un DataFrame resultado
resultados = []
for marca, datos in ventas_mensuales.groupby('marca'):
    datos = datos.sort_values('mes')
    datos = datos.set_index('mes')['venta_cop_año_ant']
    venta_ultimo_mes = datos.to_dict()
    venta_ultimo_mes = {k: venta_ultimo_mes.get(k, 0) for k in range(1, 13)}  # asegurar que estén todos los meses
    venta_diciembre = venta_ultimo_mes[12]
    venta_noviembre = venta_ultimo_mes[11]
    trimestre = [venta_ultimo_mes[9], venta_ultimo_mes[10], venta_ultimo_mes[11]]
    año_completo = [venta_ultimo_mes[m] for m in range(1, 12)]  # enero a noviembre
    var_ult_mes = ((venta_diciembre - venta_noviembre) / venta_noviembre) if venta_noviembre != 0 else None
    prom_año = sum(año_completo) / len(año_completo) if año_completo else 0
    var_prom_año = ((venta_diciembre - prom_año) / prom_año) if prom_año != 0 else None
    prom_trimestre = sum(trimestre) / len(trimestre) if trimestre else 0
    var_trimestre = ((venta_diciembre - prom_trimestre) / prom_trimestre) if prom_trimestre != 0 else None
    resultados.append({
        'marca': marca,
        'ventas_totales': sum(venta_ultimo_mes.values()),
        'var_ultimo_mes': var_ult_mes,
        'var_ultimo_trimestre': var_trimestre,
        'var_prom_anual': var_prom_año
    })
pd.DataFrame(resultados)

,marca,ventas_totales,var_ultimo_mes,var_ultimo_trimestre,var_prom_anual
0,Chocolisto,604061.00,0.141348,0.049502,-0.020626
1,Dux,383952.00,-0.094540,-0.128978,-0.053224
2,Jumbo,592990.99,0.259080,0.004891,-0.018294
3,Matiz,574141.00,0.108687,0.112625,0.168551


In [ ]:
# analisis por material de las marcas opcionadas para aplicar descuentos
lista_marcas = ['Chocolisto']

materiales_dctos =  marcas_analizar[marcas_analizar['marca'].isin(lista_marcas)]
materiales_dctos = materiales_dctos.groupby(['marca','codigo_sap','producto']).agg(
    total_ventas = ('venta_cop_año_ant','sum'),
    media_ventas = ('venta_cop_año_ant','mean'),
    desviacion_ventas = ('venta_cop_año_ant','std')
).reset_index().sort_values('total_ventas',ascending=False)
materiales_dctos

In [57]:
materiales_dctos = pd.merge(materiales_dctos,margen_material[['cod_material','margen_real']], how = 'left', left_on='codigo_sap', right_on='cod_material' )
materiales_dctos = materiales_dctos.drop(columns=['cod_material'])
materiales_dctos['margen_real_label'] = materiales_dctos['margen_real'].round(2)
materiales_dctos

,marca,codigo_sap,producto,total_ventas,media_ventas,desviacion_ventas,margen_real,margen_real_label
0,Chocolisto,1018160,Modifr CHOCOLISTO choco 44bsx200g,226283.0,18856.916667,1177.534671,59.519200,59.52
1,Chocolisto,1004163,Modifr. CHOCOLISTO 24bolx 400g,112543.0,9378.583333,909.992553,51.388141,51.39
2,Chocolisto,1053557,Modifr. CHOCOLISTO choco 12undx750g,51306.0,4275.500000,1327.310712,51.614226,51.61
3,Chocolisto,1003885,Modifr. CHOCOLISTO 12tarx1000g,33248.0,2770.666667,1254.230248,47.143761,47.14
4,Chocolisto,1038974,Modifr. CHOCOLISTO Crocante PG 250gx24,32633.0,2719.416667,352.870426,55.840589,55.84
5,Chocolisto,1053338,Modifr. CHOCOLISTO Sin azúcar 360g x24,28257.0,2354.750000,307.963435,51.167596,51.17
6,Chocolisto,1053337,Modifr. CHOCOLISTO Sin azúcar 180g x24,21880.0,1823.333333,199.884209,55.713267,55.71
7,Chocolisto,1044280,Modifr. CHOCOLISTO Crocante 275g X24EXT,21318.0,1776.500000,1157.705607,NaN,NaN
8,Chocolisto,1049849,Modifr. CHOCOLISTO choco 44bsx220g EXTRA,20152.0,1679.333333,1917.275782,NaN,NaN
9,Chocolisto,1038975,Modifr. CHOCOLISTO Crocante PG 500gx18,20015.0,1667.916667,396.724420,49.649401,49.65


In [ ]:
materiales_dctos['codigo_sap']

22